In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'      'Internship-Aug2020-Gowtham R.pdf'   Photos
'Getting started.pdf'  'My Drive'			   'Pongal 2021'


In [2]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project

/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project


In [28]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from os.path import join
from myFunctions.candle_stick_plot import plotCandleStick
from myFunctions.list_data_dir import load_data_files
from myFunctions import dataConversion 
from myFunctions.predictionPlot import prediction_plot
from sklearn.model_selection import train_test_split
from keras.layers import Dense,LSTM,SimpleRNN
from keras.models import Sequential,load_model
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor,KerasClassifier
from sklearn.model_selection import RandomizedSearchCV,KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math

In [5]:
fields = ['Date','High','Low','Open','Close']
lookback = 30
datafolder = r"/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/Data/NIFTY50"
prediction_date = pd.Timestamp(2020,11,29,00,00,00)
data_files = load_data_files(datafolder)


In [6]:
companyname = "HCLTECH"
filename = companyname + ".csv"
datafile= join(datafolder,filename) 

In [7]:
dnn_model = load_model('/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/models/high_model')
rnn_model = load_model("/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/models/high_rnn_model")
lstm_model = load_model("/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/models/high_lstm_model")

In [8]:
dnn_model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_147 (Dense)            (None, 500)               15500     
_________________________________________________________________
dense_148 (Dense)            (None, 50)                25050     
_________________________________________________________________
dense_149 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_150 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_151 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_152 (Dense)            (None, 1)                 51        
Total params: 48,251
Trainable params: 48,251
Non-trainable params: 0
_________________________________________________

In [9]:
rnn_model.summary()

Model: "sequential_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_52 (SimpleRNN)    (None, 50)                2600      
_________________________________________________________________
dense_129 (Dense)            (None, 15)                765       
_________________________________________________________________
dense_130 (Dense)            (None, 15)                240       
_________________________________________________________________
dense_131 (Dense)            (None, 1)                 16        
Total params: 3,621
Trainable params: 3,621
Non-trainable params: 0
_________________________________________________________________


In [10]:
lstm_model.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_153 (LSTM)              (None, 30, 50)            10400     
_________________________________________________________________
lstm_154 (LSTM)              (None, 30, 20)            5680      
_________________________________________________________________
lstm_155 (LSTM)              (None, 30, 20)            3280      
_________________________________________________________________
lstm_156 (LSTM)              (None, 30, 20)            3280      
_________________________________________________________________
lstm_157 (LSTM)              (None, 5)                 520       
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 6         
Total params: 23,166
Trainable params: 23,166
Non-trainable params: 0
_________________________________________________

In [11]:
df = pd.read_csv(datafile,usecols = fields)
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Open,High,Low,Close
0,2000-01-11,1550.0,1725.00,1492.00,1554.45
1,2000-01-12,1560.0,1678.85,1560.00,1678.85
2,2000-01-13,1790.0,1813.20,1781.00,1813.20
3,2000-01-14,1958.3,1958.30,1835.00,1958.30
4,2000-01-17,2115.0,2115.00,1801.65,1801.65


In [12]:
plotCandleStick(df,companyname)

In [13]:
High_X , High_y = dataConversion.convert2matrix(df['High'],lookback)

In [14]:
dnn_predicts = dnn_model.predict(High_X)

In [15]:
dnn_r2_score = r2_score(High_y,dnn_predicts)

In [19]:
High_X_reshaped = High_X.reshape(len(High_X),30,1)

In [21]:
rnn_predicts = rnn_model.predict(High_X_reshaped)
lstm_predicts = lstm_model.predict(High_X_reshaped)

In [22]:
dnn_r2_score

0.988915049384684

In [24]:
rnn_r2_score = r2_score(High_y,rnn_predicts)

In [25]:
rnn_r2_score

-2.4785824494561974

In [26]:
lstm_r2_score = r2_score(High_y,lstm_predicts)

In [27]:
lstm_r2_score

0.8555950455047183

In [29]:
dnn_mse = mean_squared_error(High_y,dnn_predicts)
dnn_rmse = math.sqrt(dnn_mse)
dnn_mae = mean_absolute_error(High_y,dnn_predicts)

In [30]:
rnn_mse = mean_squared_error(High_y,rnn_predicts)
rnn_rmse = math.sqrt(rnn_mse)
rnn_mae = mean_absolute_error(High_y,rnn_predicts)

In [31]:
lstm_mse = mean_squared_error(High_y,lstm_predicts)
lstm_rmse = math.sqrt(lstm_mse)
lstm_mae = mean_absolute_error(High_y,lstm_predicts)

In [32]:
dnn_mse

1827.8611648731048

In [33]:
rnn_mse

573603.4366616679

In [34]:
lstm_mse

23811.762225849863